In [1]:
from snp_query_box.dsnp_transform import dsnpTransform

In [2]:
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = False

In [3]:
import os
import pandas as pd

import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta


from snp_query_box import DsnpHelperFunction, snpQueries

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [4]:
print(f"User: {USERNAME} Started creating pull_hedis and hedis_ps_wide in the shared storage")

User: a845514 Started creating pull_hedis and hedis_ps_wide in the shared storage


In [5]:
start_time = time.time()

In [6]:
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)

last_month = last_date_of_last_month_str[:7]
one_year_rolling_back_date = pd.Timestamp(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1))
print(one_year_rolling_back_date)

first_date_of_year = DsnpHelperFunction.first_date_of_year(today_str, output_type="string")
print(first_date_of_year)
first_date_of_last_month_str = DsnpHelperFunction.first_date_of_last_month(today_str, output_type="string")
print(first_date_of_last_month_str)

reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

2024-05-01 00:00:00
2025-01-01
2025-04-01
2025-04-30


In [7]:
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files/2025-04-30
The folder already exist
Output files will sit in the folder above.


In [8]:
#locate data dump folder
if new_pull:
    pull_hedis = snpQueries.pull_hedis_bq(reporting_year=2025)
    pull_hedis.to_parquet(f"{output_path}/pull_hedis.parquet")
else:
    try:
        pull_hedis = pd.read_parquet(f"{output_path}/pull_hedis.parquet")
    except:
        pull_hedis = snpQueries.pull_hedis_bq(reporting_year=2025)
        pull_hedis.to_parquet(f"{output_path}/pull_hedis.parquet")


In [9]:
#TODO This should be updated in original table, KED is not starmeasure in DB
#reporting_end_date = '2024-07-31'
#storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
#output_path = f'{storage_path}/{reporting_end_date}'
#pull_hedis = pd.read_parquet(f'{output_path}/pull_hedis.parquet')
pull_hedis.loc[pull_hedis["MeasureID"]==600484, "StarMeasure"] = 1
pull_hedis.loc[pull_hedis["MeasureID"]==201185, "StarMeasure"] = 1

#We want COB measure too
pull_hedis.loc[pull_hedis["MeasureID"]==201187, "StarMeasure"] = 1
pull_hedis.loc[pull_hedis["MeasureID"]==107126, "StarMeasure"] = 1



In [10]:
pull_hedis.head()

,individualID,StarsMEMBER_ID,SourceMemberID,ReportingYear,MeasureID,MeasureDescription,Domain,StarMeasure,DENOMINATOR,NUMERATOR,EpisodeDate,ComplianceDate,CurrGapInd,PriorGapInd,CurrentRecord,CMSContractNumber,MBI,PBPID
0,99818567,1389176,101158856400,2025,201185,The percentage of individuals =65 years of age...,PartD,1,1,0,NaT,NaT,NO-GAP,N/A,1,H5521,",:mll=5F*q%",510
1,5167225326954,463325,101241127900,2025,600382,NA-Outlier Rate,HEDIS,0,1,0,NaT,NaT,N/A,N/A,1,H5522,*m4\t=GAIa(,802
2,129406571,24395496,101358355400,2025,201507,SPC-Statin Therapy for Patients with Cardiovas...,HEDIS,1,1,0,2025-04-11,NaT,GAP,N/A,1,H5521,Sc!PWGs2\Q|,170
3,6647189877297,3614270,101483340100,2025,600382,NA-Outlier Rate,HEDIS,0,1,0,NaT,NaT,N/A,N/A,1,H7301,dUS9MT/n+vq,006
4,458444305,24248730,101052636600,2025,104201,OMW-Osteoporosis Management in Women who had a...,HEDIS,1,1,1,2024-12-23,2025-03-25,NO-GAP,N/A,1,H3959,&m||4<R**&8,032


In [11]:
omw_df = pull_hedis[(pull_hedis["MeasureID"]==104201) & (pull_hedis["CurrentRecord"] == 1)]
omw_df.loc[:, "omw_due_date"] = omw_df["EpisodeDate"] + pd.DateOffset(months=6)

In [12]:
omw_df_select = omw_df[["SourceMemberID", "ReportingYear", "MeasureID", "MeasureDescription", "DENOMINATOR", "NUMERATOR", "EpisodeDate", "ComplianceDate", "CMSContractNumber", "PBPID", "omw_due_date"]]

In [13]:
omw_df_select.to_parquet(f"{output_path}/omw_df.parquet")
omw_df_select.to_csv(f"{output_path}/omw_df.csv", index=False)

In [14]:
#Prepare call log data, one year back
hedis_ps_wide = dsnpTransform.get_wide_hedis_ps(pull_hedis)

In [15]:
hedis_ps_measure_df = DsnpHelperFunction.get_hedis_patient_safety_measure(year=2025)
hedis_ps_measure_df = hedis_ps_measure_df[['MEASURE_ID','MEASURE_NM']].drop_duplicates()

In [16]:
hedis_ps_measure_df

,MEASURE_ID,MEASURE_NM
0,101803,Controlling High Blood Pressure
1,104201,Osteoporosis Management in Women who had a Fra...
2,107125,Care for Older Adults - Medication Review
3,107126,Care for Older Adults - Functional Status Asse...
4,107127,Care for Older Adults - Pain Assessment
5,201180,Medication Adherence for Cholesterol (Statins)
6,201181,Medication Adherence for Hypertension (RAS ant...
7,201182,Medication Adherence for Diabetes Medications
8,201183,Statin Use in Persons with Diabetes
9,201184,MTM Program Completion Rate for CMR


In [17]:
measure_id_list = pull_hedis.MeasureID.astype(str).drop_duplicates().to_list()

measure_not_in_db_yet = hedis_ps_measure_df[~hedis_ps_measure_df["MEASURE_ID"].isin(measure_id_list)]

In [18]:
measure_not_in_db_yet_list = measure_not_in_db_yet["MEASURE_NM"].to_list()
measure_not_in_db_yet_list[:2]

['MTM Program Completion Rate for CMR',
 'Transitions of Care (Part C) - Average']

In [19]:
measure_not_in_db_yet_list

['MTM Program Completion Rate for CMR',
 'Transitions of Care (Part C) - Average']

In [20]:
hedis_ps_wide.head()

,SourceMemberID,DENOMINATOR_Breast Cancer Screening,DENOMINATOR_Care for Older Adults - Medication Review,DENOMINATOR_Care for Older Adults - Pain Assessment,DENOMINATOR_Colorectal Cancer Screening,DENOMINATOR_Concurrent Opioid and Benzos,DENOMINATOR_Controlling High Blood Pressure,DENOMINATOR_Diabetes Care - Blood Sugar Controlled,DENOMINATOR_Diabetes Care - Eye Exam,DENOMINATOR_Follow-up after ED Visit (FMC),DENOMINATOR_Kidney Health Evaluation for Patients with Diabetes,DENOMINATOR_Medication Adherence for Cholesterol (Statins),DENOMINATOR_Medication Adherence for Diabetes Medications,DENOMINATOR_Medication Adherence for Hypertension (RAS antagonists),DENOMINATOR_Osteoporosis Management in Women who had a Fracture,DENOMINATOR_Polypharmacy Anticholinergics,DENOMINATOR_Statin Therapy for Patients with Cardiovascular Disease,DENOMINATOR_Statin Use in Persons with Diabetes,NUMERATOR_Breast Cancer Screening,NUMERATOR_Care for Older Adults - Medication Review,NUMERATOR_Care for Older Adults - Pain Assessment,NUMERATOR_Colorectal Cancer Screening,NUMERATOR_Concurrent Opioid and Benzos,NUMERATOR_Controlling High Blood Pressure,NUMERATOR_Diabetes Care - Blood Sugar Controlled,NUMERATOR_Diabetes Care - Eye Exam,NUMERATOR_Follow-up after ED Visit (FMC),NUMERATOR_Kidney Health Evaluation for Patients with Diabetes,NUMERATOR_Medication Adherence for Cholesterol (Statins),NUMERATOR_Medication Adherence for Diabetes Medications,NUMERATOR_Medication Adherence for Hypertension (RAS antagonists),NUMERATOR_Osteoporosis Management in Women who had a Fracture,NUMERATOR_Polypharmacy Anticholinergics,NUMERATOR_Statin Therapy for Patients with Cardiovascular Disease,NUMERATOR_Statin Use in Persons with Diabetes,total_denominator,total_numerator,total_open_gap,open_gap_Breast Cancer Screening,open_gap_Care for Older Adults - Medication Review,open_gap_Care for Older Adults - Pain Assessment,open_gap_Colorectal Cancer Screening,open_gap_Concurrent Opioid and Benzos,open_gap_Controlling High Blood Pressure,open_gap_Diabetes Care - Blood Sugar Controlled,open_gap_Diabetes Care - Eye Exam,open_gap_Follow-up after ED Visit (FMC),open_gap_Kidney Health Evaluation for Patients with Diabetes,open_gap_Medication Adherence for Cholesterol (Statins),open_gap_Medication Adherence for Diabetes Medications,open_gap_Medication Adherence for Hypertension (RAS antagonists),open_gap_Osteoporosis Management in Women who had a Fracture,open_gap_Polypharmacy Anticholinergics,open_gap_Statin Therapy for Patients with Cardiovascular Disease,open_gap_Statin Use in Persons with Diabetes
0,0000355373,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,0,1,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N
1,0000380821,<NA>,1,1,<NA>,<NA>,<NA>,1,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,0,0,<NA>,<NA>,<NA>,1,1,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,6,3,3,N,Y,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N
2,0000385757,<NA>,1,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3,0,3,N,Y,Y,N,N,Y,N,N,N,N,N,N,N,N,N,N,N
3,0000412086,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,2,N,Y,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N
4,0000417046,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,2,N,Y,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N


In [21]:
#add columns to avoid schema change
for col in measure_not_in_db_yet_list:
    hedis_ps_wide[f'DENOMINATOR_{col}'] = pd.NA
    hedis_ps_wide[f'NUMERATOR_{col}'] = pd.NA
    hedis_ps_wide[f'open_gap_{col}'] = 'N'

In [22]:
hedis_ps_wide["open_gap_Concurrent Opioid and Benzos"].value_counts()

open_gap_Concurrent Opioid and Benzos
N    2926297
Y     173533
Name: count, dtype: int64

In [23]:
#save it for monthly aggregation
hedis_ps_wide.to_parquet(f"{output_path}/hedis_ps_wide.parquet")

In [24]:
hedis_ps_wide["open_gap_Polypharmacy Anticholinergics"].value_counts()

open_gap_Polypharmacy Anticholinergics
N    2987300
Y     112530
Name: count, dtype: int64

In [25]:
print("Process time -- %s seconds" % (time.time() - start_time))

Process time -- 1489.9368097782135 seconds
